In [1]:
import os
import sys
import csv
import requests
from bs4 import BeautifulSoup
import json
import datetime
from datetime import date, timedelta
import pandas as pd
import numpy as np

# News data

In [2]:
today = date.today()



In [ ]:
'''
api = "16ac337f8ef1480c898f80990299d106"
url = "https://newsapi.org/v2/everything"

while True:

    today = today
    
    end = date(2018, 2, 1)

    if today >= end:

      next = today - timedelta(days=7)

      params = {
          "q": "crypto AND (bitcoin OR btc)",
          "from": today,
          "to": next,
          "language": "en",
          "sortBy": "relevancy",
          "apiKey": api
      }


      response = requests.get(url, params=params)
      news = []
      time = []

      if response.status_code == 200:
          data = response.json()
          for des in data["articles"]:
              time.append(des["publishedAt"])
              news.append(des["description"])

          with open("btcnews.csv", mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Time", "News"])
            for t, n in zip(time, news):
              writer.writerow([t, n])

          print(f"{today} to {next} done...")

      else:
          print("Failed to fetch news. Status code:", response.status_code)

      

      today = today - timedelta(days=7)

    else:
      break

'''


# Fear and Greed

In [11]:
url = "https://api.alternative.me/fng/?limit=0"
csv_file = "btcfg.csv"
response = requests.get(url)
data = response.json()
index_data = data['data']

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Timestamp', 'Value', 'Value_classification'])
    for entry in index_data:
        timestamp = datetime.datetime.utcfromtimestamp(int(entry['timestamp']))
        value = entry['value']
        value_classification = entry['value_classification']
        writer.writerow([timestamp, value, value_classification])

print(f"Data written")


Data written


# BTC Price

In [3]:
import requests

def get_btc_price():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        btc_price = data["bitcoin"]["usd"]
        return btc_price
    else:
        print("Failed to fetch data")
        return None

# Example usage
btc_price = get_btc_price()
if btc_price is not None:
    print("Current BTC price: $", btc_price)


Current BTC price: $ 68449


# Data preprocessing

## BTCUSD

In [4]:
btc = pd.read_csv("../data/2015/btcusd.csv")

In [5]:
btc.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [6]:
btc.describe()

,Date,Price,Open,High,Low,Vol.,Change %
count,3413,3413,3413,3413,3413,3403,3413
unique,3413,3295,3282,3252,3263,2052,1257
top,05/07/2024,604.1,604.1,604.1,604.1,1.44K,0.00%
freq,1,7,8,7,7,9,18


In [7]:
btc.rename(columns={'Price': 'btc_price', 'Open':'btc_open', 'High':'btc_high', 'Low':'btc_low', 'Vol.':'btc_vol', 'Change %':'btc_change'}, inplace=True)

In [8]:
btc.isna().sum()

Date           0
btc_price      0
btc_open       0
btc_high       0
btc_low        0
btc_vol       10
btc_change     0
dtype: int64

In [9]:
btc['Date'] = pd.to_datetime(btc['Date'], format='%m/%d/%Y')

In [10]:
btc.drop(columns=['btc_vol'], inplace=True)

In [11]:
btc.head()

,Date,btc_price,btc_open,btc_high,btc_low,btc_change
0,2024-05-07,"63,753.0","63,280.0","64,480.0","62,987.0",0.75%
1,2024-05-06,"63,280.0","64,173.0","65,625.0","62,812.0",-1.39%
2,2024-05-05,"64,173.0","63,983.0","64,769.0","63,034.0",0.30%
3,2024-05-04,"63,983.0","62,978.0","64,644.0","62,690.0",1.60%
4,2024-05-03,"62,978.0","59,247.0","63,429.0","58,914.0",6.30%


In [12]:
btc['btc_change'] = btc['btc_change'].str.replace('%', '')

In [13]:
cols = ['btc_price', 'btc_open', 'btc_high', 'btc_low']

for i in range(len(cols)):
    btc[cols[i]] = btc[cols[i]].str.replace(',', '')

In [14]:
btc['btc_price'] = pd.to_numeric(btc['btc_price'], errors='coerce')
btc['btc_open'] = pd.to_numeric(btc['btc_open'], errors='coerce')
btc['btc_high'] = pd.to_numeric(btc['btc_high'], errors='coerce')
btc['btc_low'] = pd.to_numeric(btc['btc_low'], errors='coerce')
btc['btc_change'] = pd.to_numeric(btc['btc_change'], errors='coerce')
btc.dtypes


Date          datetime64[ns]
btc_price            float64
btc_open             float64
btc_high             float64
btc_low              float64
btc_change           float64
dtype: object

In [15]:
btc.head()

,Date,btc_price,btc_open,btc_high,btc_low,btc_change
0,2024-05-07,63753.0,63280.0,64480.0,62987.0,0.75
1,2024-05-06,63280.0,64173.0,65625.0,62812.0,-1.39
2,2024-05-05,64173.0,63983.0,64769.0,63034.0,0.30
3,2024-05-04,63983.0,62978.0,64644.0,62690.0,1.60
4,2024-05-03,62978.0,59247.0,63429.0,58914.0,6.30


In [16]:
btc.isna().sum()

Date          0
btc_price     0
btc_open      0
btc_high      0
btc_low       0
btc_change    0
dtype: int64

# Gold

In [17]:
gold = pd.read_csv("../data/2015/gold.csv")

In [18]:
gold.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,05/07/2024,"2,327.60","2,334.60","2,338.65","2,318.30",NaN,-0.15%
1,05/06/2024,"2,331.20","2,312.80","2,341.90","2,300.60",191.90K,0.98%
2,05/03/2024,"2,308.60","2,313.50","2,330.70","2,285.20",231.40K,-0.04%
3,05/02/2024,"2,309.60","2,329.90","2,336.10","2,294.30",193.66K,-0.06%
4,05/01/2024,"2,311.00","2,298.20","2,339.50","2,291.70",227.38K,0.35%


In [19]:
gold.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [20]:
gold.describe()

,Date,Price,Open,High,Low,Vol.,Change %
count,2422,2422,2422,2422,2422,2400,2422
unique,2422,2049,2005,2066,2046,2241,435
top,05/07/2024,"1,208.20","1,345.00","1,288.40","1,246.90",0.37K,0.00%
freq,1,5,5,4,4,6,26


In [21]:
gold.rename(columns={'Price': 'gold_price', 'Open':'gold_open', 'High':'gold_high', 'Low':'gold_low', 'Vol.':'gold_vol', 'Change %':'gold_change'}, inplace=True)

In [22]:
gold.isna().sum()

Date            0
gold_price      0
gold_open       0
gold_high       0
gold_low        0
gold_vol       22
gold_change     0
dtype: int64

In [23]:
gold['Date'] = pd.to_datetime(gold['Date'], format='%m/%d/%Y')

In [24]:
gold.drop(columns=['gold_vol'], inplace=True)

In [25]:
gold.head()

,Date,gold_price,gold_open,gold_high,gold_low,gold_change
0,2024-05-07,"2,327.60","2,334.60","2,338.65","2,318.30",-0.15%
1,2024-05-06,"2,331.20","2,312.80","2,341.90","2,300.60",0.98%
2,2024-05-03,"2,308.60","2,313.50","2,330.70","2,285.20",-0.04%
3,2024-05-02,"2,309.60","2,329.90","2,336.10","2,294.30",-0.06%
4,2024-05-01,"2,311.00","2,298.20","2,339.50","2,291.70",0.35%


In [26]:
gold['gold_change'] = gold['gold_change'].str.replace('%', '')

cols = ['gold_price', 'gold_open', 'gold_high', 'gold_low']

for i in range(len(cols)):
    gold[cols[i]] = gold[cols[i]].str.replace(',', '')

gold['gold_price'] = pd.to_numeric(gold['gold_price'], errors='coerce')
gold['gold_open'] = pd.to_numeric(gold['gold_open'], errors='coerce')
gold['gold_high'] = pd.to_numeric(gold['gold_high'], errors='coerce')
gold['gold_low'] = pd.to_numeric(gold['gold_low'], errors='coerce')
gold['gold_change'] = pd.to_numeric(gold['gold_change'], errors='coerce')
gold.dtypes


Date           datetime64[ns]
gold_price            float64
gold_open             float64
gold_high             float64
gold_low              float64
gold_change           float64
dtype: object

In [27]:
gold.head()

,Date,gold_price,gold_open,gold_high,gold_low,gold_change
0,2024-05-07,2327.6,2334.6,2338.65,2318.3,-0.15
1,2024-05-06,2331.2,2312.8,2341.90,2300.6,0.98
2,2024-05-03,2308.6,2313.5,2330.70,2285.2,-0.04
3,2024-05-02,2309.6,2329.9,2336.10,2294.3,-0.06
4,2024-05-01,2311.0,2298.2,2339.50,2291.7,0.35


# S&P500

In [28]:
sp = pd.read_csv("../data/2015/sp500.csv")

In [29]:
sp.dtypes

Date         object
Price        object
Open         object
High         object
Low          object
Vol.        float64
Change %     object
dtype: object

In [30]:
sp.drop(columns=["Vol."], inplace=True)

In [31]:
sp.head()

,Date,Price,Open,High,Low,Change %
0,05/06/2024,"5,180.74","5,142.42","5,181.00","5,142.42",1.03%
1,05/03/2024,"5,127.79","5,122.78","5,139.12","5,101.22",1.26%
2,05/02/2024,"5,064.20","5,049.32","5,073.21","5,011.05",0.91%
3,05/01/2024,"5,018.39","5,029.03","5,096.12","5,013.45",-0.34%
4,04/30/2024,"5,035.69","5,103.78","5,110.83","5,035.31",-1.57%


In [32]:
sp.describe()

,Date,Price,Open,High,Low,Change %
count,2351,2351,2351,2351,2351,2351
unique,2351,2224,2208,2207,2235,498
top,05/06/2024,"2,268.90","2,110.60","2,094.30","2,119.90",-0.20%
freq,1,3,3,3,3,21


In [33]:
sp.rename(columns={'Price': 'sp_price', 'Open':'sp_open', 'High':'sp_high', 'Low':'sp_low', 'Change %':'sp_change'}, inplace=True)

In [34]:
sp.isna().sum()

Date         0
sp_price     0
sp_open      0
sp_high      0
sp_low       0
sp_change    0
dtype: int64

In [35]:
sp['Date'] = pd.to_datetime(sp['Date'], format='%m/%d/%Y')

In [36]:
sp.head()

,Date,sp_price,sp_open,sp_high,sp_low,sp_change
0,2024-05-06,"5,180.74","5,142.42","5,181.00","5,142.42",1.03%
1,2024-05-03,"5,127.79","5,122.78","5,139.12","5,101.22",1.26%
2,2024-05-02,"5,064.20","5,049.32","5,073.21","5,011.05",0.91%
3,2024-05-01,"5,018.39","5,029.03","5,096.12","5,013.45",-0.34%
4,2024-04-30,"5,035.69","5,103.78","5,110.83","5,035.31",-1.57%


In [37]:
sp['sp_change'] = sp['sp_change'].str.replace('%', '')

cols = ['sp_price', 'sp_open', 'sp_high', 'sp_low']

for i in range(len(cols)):
    sp[cols[i]] = sp[cols[i]].str.replace(',', '')

sp['sp_price'] = pd.to_numeric(sp['sp_price'], errors='coerce')
sp['sp_open'] = pd.to_numeric(sp['sp_open'], errors='coerce')
sp['sp_high'] = pd.to_numeric(sp['sp_high'], errors='coerce')
sp['sp_low'] = pd.to_numeric(sp['sp_low'], errors='coerce')
sp['sp_change'] = pd.to_numeric(sp['sp_change'], errors='coerce')
sp.dtypes


Date         datetime64[ns]
sp_price            float64
sp_open             float64
sp_high             float64
sp_low              float64
sp_change           float64
dtype: object

In [38]:
sp.head()

,Date,sp_price,sp_open,sp_high,sp_low,sp_change
0,2024-05-06,5180.74,5142.42,5181.00,5142.42,1.03
1,2024-05-03,5127.79,5122.78,5139.12,5101.22,1.26
2,2024-05-02,5064.20,5049.32,5073.21,5011.05,0.91
3,2024-05-01,5018.39,5029.03,5096.12,5013.45,-0.34
4,2024-04-30,5035.69,5103.78,5110.83,5035.31,-1.57


# US30

In [39]:
us30 = pd.read_csv("../data/2015/us30.csv")

In [40]:
us30.dtypes

Date        object
Price       object
Open        object
High        object
Low         object
Vol.        object
Change %    object
dtype: object

In [41]:
us30.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,05/07/2024,"38,908.4","38,847.9","38,945.4","38,838.5",NaN,0.16%
1,05/06/2024,"38,847.9","38,722.5","38,880.0","38,689.3",NaN,0.30%
2,05/05/2024,"38,730.2","38,757.5","38,765.0","38,717.5",NaN,0.06%
3,05/03/2024,"38,706.4","38,457.4","38,804.0","38,423.1",NaN,0.65%
4,05/02/2024,"38,457.9","38,026.5","38,531.7","37,879.2",NaN,1.17%


In [42]:
us30.drop(columns=['Vol.'], inplace=True)

In [43]:
us30.describe()

,Date,Price,Open,High,Low,Change %
count,2601,2601,2601,2601,2601,2601
unique,2601,2577,2581,2587,2573,471
top,05/07/2024,"18,086.4","33,625.5","34,472.5","26,548.7",-0.10%
freq,1,2,2,2,2,27


In [44]:
us30.rename(columns={'Price': 'us30_price', 'Open':'us30_open', 'High':'us30_high', 'Low':'us30_low', 'Change %':'us30_change'}, inplace=True)

In [45]:
us30.isna().sum()

Date           0
us30_price     0
us30_open      0
us30_high      0
us30_low       0
us30_change    0
dtype: int64

In [46]:
us30['Date'] = pd.to_datetime(us30['Date'], format='%m/%d/%Y')

In [47]:
us30.head()

,Date,us30_price,us30_open,us30_high,us30_low,us30_change
0,2024-05-07,"38,908.4","38,847.9","38,945.4","38,838.5",0.16%
1,2024-05-06,"38,847.9","38,722.5","38,880.0","38,689.3",0.30%
2,2024-05-05,"38,730.2","38,757.5","38,765.0","38,717.5",0.06%
3,2024-05-03,"38,706.4","38,457.4","38,804.0","38,423.1",0.65%
4,2024-05-02,"38,457.9","38,026.5","38,531.7","37,879.2",1.17%


In [48]:
us30['us30_change'] = us30['us30_change'].str.replace('%', '')

cols = ['us30_price', 'us30_open', 'us30_high', 'us30_low']

for i in range(len(cols)):
    us30[cols[i]] = us30[cols[i]].str.replace(',', '')

us30['us30_price'] = pd.to_numeric(us30['us30_price'], errors='coerce')
us30['us30_open'] = pd.to_numeric(us30['us30_open'], errors='coerce')
us30['us30_high'] = pd.to_numeric(us30['us30_high'], errors='coerce')
us30['us30_low'] = pd.to_numeric(us30['us30_low'], errors='coerce')
us30['us30_change'] = pd.to_numeric(us30['us30_change'], errors='coerce')
us30.dtypes


Date           datetime64[ns]
us30_price            float64
us30_open             float64
us30_high             float64
us30_low              float64
us30_change           float64
dtype: object

In [49]:
us30.head()

,Date,us30_price,us30_open,us30_high,us30_low,us30_change
0,2024-05-07,38908.4,38847.9,38945.4,38838.5,0.16
1,2024-05-06,38847.9,38722.5,38880.0,38689.3,0.30
2,2024-05-05,38730.2,38757.5,38765.0,38717.5,0.06
3,2024-05-03,38706.4,38457.4,38804.0,38423.1,0.65
4,2024-05-02,38457.9,38026.5,38531.7,37879.2,1.17


# USD Index

In [50]:
usidx = pd.read_csv("../data/2015/usdindex.csv")

In [51]:
usidx.dtypes

Date         object
Price       float64
Open        float64
High        float64
Low         float64
Vol.         object
Change %     object
dtype: object

In [52]:
usidx.isnull().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64

In [53]:
usidx.drop(columns=['Vol.'], inplace=True)

In [54]:
usidx.head()

,Date,Price,Open,High,Low,Change %
0,"May 07, 2024",104.955,105.020,105.180,104.915,-0.00%
1,"May 06, 2024",104.960,105.065,105.080,104.745,-0.10%
2,"May 05, 2024",105.060,104.940,105.065,104.940,0.10%
3,"May 03, 2024",104.960,105.180,105.250,104.410,-0.21%
4,"May 02, 2024",105.178,105.590,105.785,105.170,-0.43%


In [55]:
usidx.rename(columns={'Price': 'usidx_price', 'Open':'usidx_open', 'High':'usidx_high', 'Low':'usidx_low', 'Change %':'usidx_change'}, inplace=True)

In [56]:
usidx['Date'] = pd.to_datetime(usidx['Date'], format='%b %d, %Y')

In [57]:
usidx.head()

,Date,usidx_price,usidx_open,usidx_high,usidx_low,usidx_change
0,2024-05-07,104.955,105.020,105.180,104.915,-0.00%
1,2024-05-06,104.960,105.065,105.080,104.745,-0.10%
2,2024-05-05,105.060,104.940,105.065,104.940,0.10%
3,2024-05-03,104.960,105.180,105.250,104.410,-0.21%
4,2024-05-02,105.178,105.590,105.785,105.170,-0.43%


In [58]:
usidx['usidx_change'] = usidx['usidx_change'].str.replace('%', '')

'''
cols = ['usidx_price', 'usidx_open', 'usidx_high', 'usidx_low']

for i in range(len(cols)):
    usidx[cols[i]] = usidx[cols[i]].str.replace(',', '')
'''

usidx['usidx_price'] = pd.to_numeric(usidx['usidx_price'], errors='coerce')
usidx['usidx_open'] = pd.to_numeric(usidx['usidx_open'], errors='coerce')
usidx['usidx_high'] = pd.to_numeric(usidx['usidx_high'], errors='coerce')
usidx['usidx_low'] = pd.to_numeric(usidx['usidx_low'], errors='coerce')
usidx['usidx_change'] = pd.to_numeric(usidx['usidx_change'], errors='coerce')
usidx.dtypes


Date            datetime64[ns]
usidx_price            float64
usidx_open             float64
usidx_high             float64
usidx_low              float64
usidx_change           float64
dtype: object

In [59]:
usidx.head()

,Date,usidx_price,usidx_open,usidx_high,usidx_low,usidx_change
0,2024-05-07,104.955,105.020,105.180,104.915,-0.00
1,2024-05-06,104.960,105.065,105.080,104.745,-0.10
2,2024-05-05,105.060,104.940,105.065,104.940,0.10
3,2024-05-03,104.960,105.180,105.250,104.410,-0.21
4,2024-05-02,105.178,105.590,105.785,105.170,-0.43


# cleaning

In [60]:
concatenated_df = pd.concat([btc.set_index('Date'),
                             gold.set_index('Date'),
                             sp.set_index('Date'),
                             us30.set_index('Date'),
                             usidx.set_index('Date')],
                            axis=1, join='outer')

In [61]:
concatenated_df = concatenated_df.reset_index()

In [62]:
concatenated_df.shape

(3414, 26)

In [63]:
concatenated_df.isna().sum()

Date               0
btc_price          1
btc_open           1
btc_high           1
btc_low            1
btc_change         1
gold_price       992
gold_open        992
gold_high        992
gold_low         992
gold_change      992
sp_price        1063
sp_open         1063
sp_high         1063
sp_low          1063
sp_change       1063
us30_price       813
us30_open        813
us30_high        813
us30_low         813
us30_change      813
usidx_price      999
usidx_open       999
usidx_high       999
usidx_low        999
usidx_change     999
dtype: int64

In [64]:
concatenated_df.head()

,Date,btc_price,btc_open,btc_high,btc_low,btc_change,gold_price,gold_open,gold_high,gold_low,...,us30_price,us30_open,us30_high,us30_low,us30_change,usidx_price,usidx_open,usidx_high,usidx_low,usidx_change
0,2015-01-01,313.9,317.5,317.5,312.9,-0.99,NaN,NaN,NaN,NaN,...,17822.0,17757.0,17825.5,17757.0,-0.01,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,315.1,313.6,316.0,313.0,0.40,1186.2,1184.0,1194.9,1167.3,...,17833.0,17823.1,17951.8,17731.3,0.06,91.383,90.81,91.49,90.80,0.81
2,2015-01-03,282.0,314.8,314.8,282.0,-10.51,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,258.8,282.0,288.9,258.8,-8.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,273.2,261.0,276.5,260.5,5.57,1204.0,1187.8,1207.5,1177.8,...,17501.7,17821.3,17821.3,17475.9,-1.86,91.622,91.73,92.05,91.55,0.26


In [65]:
df_filled_forward = concatenated_df.fillna(method='ffill')

In [66]:
df_filled = df_filled_forward.fillna(method='bfill')

In [67]:
df_filled.isna().sum()

Date            0
btc_price       0
btc_open        0
btc_high        0
btc_low         0
btc_change      0
gold_price      0
gold_open       0
gold_high       0
gold_low        0
gold_change     0
sp_price        0
sp_open         0
sp_high         0
sp_low          0
sp_change       0
us30_price      0
us30_open       0
us30_high       0
us30_low        0
us30_change     0
usidx_price     0
usidx_open      0
usidx_high      0
usidx_low       0
usidx_change    0
dtype: int64

In [68]:
df_filled.head()

,Date,btc_price,btc_open,btc_high,btc_low,btc_change,gold_price,gold_open,gold_high,gold_low,...,us30_price,us30_open,us30_high,us30_low,us30_change,usidx_price,usidx_open,usidx_high,usidx_low,usidx_change
0,2015-01-01,313.9,317.5,317.5,312.9,-0.99,1186.2,1184.0,1194.9,1167.3,...,17822.0,17757.0,17825.5,17757.0,-0.01,91.383,90.81,91.49,90.80,0.81
1,2015-01-02,315.1,313.6,316.0,313.0,0.40,1186.2,1184.0,1194.9,1167.3,...,17833.0,17823.1,17951.8,17731.3,0.06,91.383,90.81,91.49,90.80,0.81
2,2015-01-03,282.0,314.8,314.8,282.0,-10.51,1186.2,1184.0,1194.9,1167.3,...,17833.0,17823.1,17951.8,17731.3,0.06,91.383,90.81,91.49,90.80,0.81
3,2015-01-04,258.8,282.0,288.9,258.8,-8.23,1186.2,1184.0,1194.9,1167.3,...,17833.0,17823.1,17951.8,17731.3,0.06,91.383,90.81,91.49,90.80,0.81
4,2015-01-05,273.2,261.0,276.5,260.5,5.57,1204.0,1187.8,1207.5,1177.8,...,17501.7,17821.3,17821.3,17475.9,-1.86,91.622,91.73,92.05,91.55,0.26


In [72]:
df_filled.to_csv('../data/processed_data.csv', index=False)